In [2]:
import shutil
import os.path
import xarray as xr
from tests.helpers import make_test_dataset
from zappend.api import zappend

In [3]:
slice_paths = []
for index in range(10):
    slice_path = f"testdata/slice-{index}.zarr" 
    slice_paths.append(slice_path)
    make_test_dataset(uri=slice_path, 
                      shape=(1, 200, 200), 
                      chunks=(1, 200, 200),
                      index=index)

In [4]:
slice_ds = xr.open_zarr("testdata/slice-0.zarr")
slice_ds

<xarray.Dataset>
Dimensions:  (time: 1, y: 200, x: 200)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01
  * x        (x) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
  * y        (y) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
Data variables:
    chl      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>
    tsm      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>

In [5]:
slice_ds.time.encoding

{'chunks': (1,),
 'preferred_chunks': {'time': 1},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'days since 2024-01-01 00:00:00',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}

In [10]:
config = {
    "target_uri": "testdata/target.zarr", 
    "variables": {
        "time": {
            "encoding": {
                "chunks": [100]
            }
        }
    },
    "logging": {
        "version": 1,
        "formatters": {
            "normal": {
                "format": "%(asctime)s %(levelname)s %(message)s",
                "style": "%"
            }
        },
        "handlers": {
            "console": {
                "class": "logging.StreamHandler",
                "formatter": "normal"
            }
        },
        "loggers": {
            "zappend": {
                "level": "INFO",
                "handlers": ["console"]
            }
        }
    }
}

In [11]:
target_path = "testdata/target.zarr"
if os.path.exists(target_path):
    shutil.rmtree(target_path)

In [12]:
zappend(slice_paths, config=config)

2024-01-03 19:11:35,731 INFO Target dataset testdata/target.zarr not found
2024-01-03 19:11:35,731 INFO Opening slice from testdata/slice-0.zarr
2024-01-03 19:11:35,867 INFO Target dataset ftestdata/target.zarr found
2024-01-03 19:11:35,867 INFO Opening slice from testdata/slice-1.zarr
2024-01-03 19:11:35,903 INFO Consolidating target dataset
2024-01-03 19:11:36,003 INFO Target dataset ftestdata/target.zarr found
2024-01-03 19:11:36,003 INFO Opening slice from testdata/slice-2.zarr
2024-01-03 19:11:36,048 INFO Consolidating target dataset
2024-01-03 19:11:36,138 INFO Target dataset ftestdata/target.zarr found
2024-01-03 19:11:36,138 INFO Opening slice from testdata/slice-3.zarr
2024-01-03 19:11:36,175 INFO Consolidating target dataset
2024-01-03 19:11:36,256 INFO Target dataset ftestdata/target.zarr found
2024-01-03 19:11:36,268 INFO Opening slice from testdata/slice-4.zarr
2024-01-03 19:11:36,311 INFO Consolidating target dataset
2024-01-03 19:11:36,402 INFO Target dataset ftestdata/t

In [9]:
target_ds = xr.open_zarr("testdata/target.zarr")
target_ds

<xarray.Dataset>
Dimensions:  (time: 10, y: 200, x: 200)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-10
  * x        (x) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
  * y        (y) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
Data variables:
    chl      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>
    tsm      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>

In [8]:
target_ds.time

<xarray.DataArray 'time' (time: 10)>
array(['2024-01-01T00:00:00.000000000', '2024-01-02T00:00:00.000000000',
       '2024-01-03T00:00:00.000000000', '2024-01-04T00:00:00.000000000',
       '2024-01-05T00:00:00.000000000', '2024-01-06T00:00:00.000000000',
       '2024-01-07T00:00:00.000000000', '2024-01-08T00:00:00.000000000',
       '2024-01-09T00:00:00.000000000', '2024-01-10T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-10

In [9]:
target_ds.time.encoding

{'chunks': (100,),
 'preferred_chunks': {'time': 100},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'days since 2024-01-01',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}